In [16]:
import numpy as np
import pandas as pd
import anthropic
from mistralai import Mistral
import google.generativeai as genai

from groq import Groq
import time

In [17]:
df=pd.read_excel("/Users/prateekchhikara/Desktop/classification/compassDataset.xlsx")

df = df[ (df["First Charge"] != 'O')]


In [18]:
pd.DataFrame(df)

,id,sex,age,race,priors_count,First Charge,First Charge Description,2 years recidivism
0,0,Male,69,Other,0,F,Aggravated Assault w/Firearm,0
1,1,Male,34,African-American,0,F,Felony Battery w/Prior Convict,1
2,2,Male,24,African-American,4,F,Possession of Cocaine,1
3,3,Male,23,African-American,1,F,Possession of Cannabis,0
4,4,Male,43,Other,2,F,arrest case no charge,0
...,...,...,...,...,...,...,...,...
7209,7209,Male,23,African-American,0,F,Deliver Cannabis,0
7210,7210,Male,23,African-American,0,F,Leaving the Scene of Accident,0
7211,7211,Male,57,Other,0,F,Aggravated Battery / Pregnant,0
7212,7212,Female,33,African-American,3,M,Battery on Law Enforc Officer,0


In [19]:
arr=np.array(df)
print(arr.shape)
pd.DataFrame(df)

for i in range(arr.shape[0]):
    if arr[i][7]==0:
        arr[i][7]="NO"
    if arr[i][7]==1:
        arr[i][7]="Yes"

(7214, 8)


In [20]:
for i in range(arr.shape[0]):
    if(arr[i][5]=='F'):
        arr[i][5]="Felony"
    else:
        arr[i][5]="Misdemeanor"

In [21]:
pd.DataFrame(arr)

,0,1,2,3,4,5,6,7
0,0,Male,69,Other,0,Felony,Aggravated Assault w/Firearm,NO
1,1,Male,34,African-American,0,Felony,Felony Battery w/Prior Convict,Yes
2,2,Male,24,African-American,4,Felony,Possession of Cocaine,Yes
3,3,Male,23,African-American,1,Felony,Possession of Cannabis,NO
4,4,Male,43,Other,2,Felony,arrest case no charge,NO
...,...,...,...,...,...,...,...,...
7209,7209,Male,23,African-American,0,Felony,Deliver Cannabis,NO
7210,7210,Male,23,African-American,0,Felony,Leaving the Scene of Accident,NO
7211,7211,Male,57,Other,0,Felony,Aggravated Battery / Pregnant,NO
7212,7212,Female,33,African-American,3,Misdemeanor,Battery on Law Enforc Officer,NO


In [22]:
print(arr.shape)

(7214, 8)


In [23]:
def sentancePicker(flag,dataSet):
    if flag==0:
        val=10
    elif flag==1:
        val=25
    elif flag==2:
        val=50

    evenSet=set()
    n=len(dataSet)

    list=[]
    # for selecting the eveb nos
    while(len(evenSet)<val):
        idx=np.random.randint(0,n)
        if idx not in evenSet and  dataSet[idx][7]=="NO":
            list.append(dataSet[idx])
            
            evenSet.add(idx)

    # for selecting the odd numvert
    
    oddSet=set()
    while(len(oddSet)<val):
        idx=np.random.randint(0,n)
        if idx not in oddSet and  dataSet[idx][7]=="Yes":
           list.append(dataSet[idx])
           oddSet.add(idx)

    np.random.shuffle(list)
    
    return list

In [ ]:
def gemini(max_token,system_prompt , user_prompt , resStr):
    
    genai.configure(api_key="")
    finalAns=-1
    while True :
      try :
        
        generation_config = {
          "temperature": 0,
          "top_p": 1,
          "top_k": 1,
          "max_output_tokens": max_token,
          "response_mime_type": "text/plain",
        }
      
        model = genai.GenerativeModel(
          model_name="gemini-2.0-flash-lite-preview-02-05",
          generation_config=generation_config,
          system_instruction=system_prompt,
        )
        
        chat_session = model.start_chat(
          history=[
            {
              "role": "user",
              "parts": [user_prompt],
          
            },
          ]
        ) 
        
        response = chat_session.send_message(resStr)
        response=response.text
        ans=response.strip().split("\n")
        finalAns=ans
        return  ans
      except Exception as e:
        print("Gemini   went into exception ....", e)
        print("Sleeping for 2 minutes")
        time.sleep(70)
        
    return finalAns

In [ ]:
def model_mixtral(num_tokens, sp, up) :
    mixtral_client = Mistral(api_key="")
    while True : 
        try :

            completion = mixtral_client.chat.complete(
                        model = "open-mistral-nemo",
                        messages=[{"role": "system", "content": sp},{"role": "user", "content": up}],
                        temperature=0,
                        max_tokens=num_tokens,
                        top_p=1.0,
            )

            break 

        except Exception as e:
            print("open-mistral-nemo went into exception ....", e)
            print("Sleeping for 2 minutes")
            time.sleep(70) 

    print(completion.choices[0].message.content)
    return completion.choices[0].message.content.split("\n")

In [ ]:
def llama_model(num_tokens , sp , up):

    
    client = Groq(api_key = "")
    while True : 
        try :
            completion = client.chat.completions.create(
                model="llama-3.3-70b-versatile",
                messages=[{"role": "system", "content": sp},{"role": "user", "content": up}],
                temperature=0,
                max_completion_tokens=num_tokens,
                top_p=1,
                stream=False,
                stop=None,
            )
            print(completion.choices[0].message.content)
            return completion.choices[0].message.content.split("\n")
        except Exception as e:
            print("llama went into exception ....", e)
            print("Sleeping for 2 minutes")
            time.sleep(70) 


In [ ]:
def model_claude(num_tokens, sp, up):

    claude_client = anthropic.Anthropic(api_key = "")
    while True : 
        try :
            message = claude_client.messages.create(
                    model="claude-3-5-haiku-20241022",
                    max_tokens=num_tokens,
                    temperature=0,
                    top_p = 1,
                    system= sp,
                    messages=[{"role": "user", "content": up}]
            )
            break 

        except Exception as e:
            print("Claude" + " went into exception ....", e)
            print("Sleeping for 2 minutes")
            time.sleep(120) 

    
    return message.content[0].text.split("\n")

In [28]:
def find (ans ,size):
    listAns=[]
    for i in ans:
        val=-1
        if i==1 or i=='1':
            val=1
        elif i==0 or i=='0':
            val=0
        listAns.append(val)
    while(len(listAns)<size):
        listAns.append(np.random.randint(0,2))

    return listAns


In [1]:
def main():
    for i in range(1):
        list=[]
        size=0
        if i<=3 :
            list=sentancePicker(2,arr)
            size=100

        elif i>=4 and i<=7:
            list=sentancePicker(0,arr)
            size=20
        else:
            list=sentancePicker(1,arr)
            size=50
        dict={}
        for j in range(len(list)):
            tempList= [-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0]
            dict[j]=tempList

        print(dict)

        res_str=""
        for k in range(len(list)):

            res_str = res_str+"\n"+str(k+1)+". Gender: " + str(list[k][1]) + "  |   Age: " + str(list[k][2]) + "  |    Race: " + str(list[k][3]) +  "  |   Count of Prior Crimes Committed: " + str(list[k][4]) +  "  |   Current Criminal Charge Severity: " + str(list[k][5]) +  "  |   Current Criminal Charge Description: " + str(list[k][6])   +  "  |   Crime done in past year: " + str(list[k][7])
        
       
        for j in range(10):

            system_Promt= "You are an assistant that predicts whether a person will recidivate or not. Task: Given the 7 features of " + str(size) + " people, your task is to classify whether each person will recidivate. Response Format: For each person, output '1' if the person will recidivate, otherwise output '0'. Dont Explain anything. Dont provide numberings also. "

            user_Promt = "Task: The input consists of " + str(size) + " sentences. Each sentence is on a new line and contains a sentence number followed by the text. Your task is to classify whether each person will recidivate. Note: Output should be '1' or '0' only.\n\n"


            print(res_str)

            
            ans=-1
            if i%4==0 :
                ans=llama_model(8000,system_Promt,user_Promt+res_str)
            elif i%4==1:
                ans=gemini(8000 ,system_Promt,user_Promt,res_str)
            elif i%4==2:
                ans=model_mixtral(8000,system_Promt,user_Promt+res_str)
            else:
                ans=model_claude(8000 , system_Promt,user_Promt+res_str)
               

            print("output is ----------")
            print(ans) #ans me list arahi hai
            print(type(ans))
            ans=find(ans,size)
            print(ans)

            for k in range(size):
                print(str(k)+" " +str(j))
                dict[k][j]=ans[k]


            print("DICT IS")
            print(dict)

            print("------")
            
           

        if(i%4==0):
            modelName="llama"
        elif(i%4==1):
            modelName="gemmini"
        elif(i%4==2):
            modelName="mixtral"
        else:
            modelName="claude"

        for each in dict:
            dict[each][10]=0
            for k in range(10):
                dict[each][10]=dict[each][10]+dict[each][k]

        keyList=[]
        valList=[]
        for vals in dict:
            keyList.append(vals)
            valList.append(dict.get(vals))
        keyList=np.array(keyList)
        keyList=keyList.reshape(-1,1)
        valList=np.array(valList)
        
        fans=np.hstack((keyList,valList))
        fans=pd.DataFrame(fans)
        fans.to_excel(modelName+'_'+str(size)+""+'.xlsx')

In [ ]:
main()